# Breast cancer Wisconsin

## Librairies utiles

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

## Le dataset Breast Cancer Wisconsin

Le dataset est accessible sur :  
https://www.kaggle.com/uciml/breast-cancer-wisconsin-data  
http://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+%28diagnostic%29  
(on peut utiliser pd.read_table pour lire un fichier .dat)

In [ ]:
df = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')

On peut afficher les 10 premières lignes du dataset :

In [ ]:
df.head(10)

On a les informations suivantes :
1) ID number 2) Diagnosis (M = malignant, B = benign) 3-32)

Ten real-valued features are computed for each cell nucleus:

a) radius (mean of distances from center to points on the perimeter)  
b) texture (standard deviation of gray-scale values)  
c) perimeter  
d) area  
e) smoothness (local variation in radius lengths)  
f) compactness (perimeter^2 / area - 1.0)  
g) concavity (severity of concave portions of the contour)  
h) concave points (number of concave portions of the contour)  
i) symmetry  
j) fractal dimension ("coastline approximation" - 1)  
  
The mean, standard error and "worst" or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.  
  
All feature values are recoded with four significant digits.

In [ ]:
df.columns

Pour avoir l'ensemble du tableau, on peut utiliser un affichage au format HTML :

In [ ]:
from IPython.core.display import HTML # permet d'afficher du code html dans jupyter
display(HTML(df.head(10).to_html()))

## Données

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.columns

La colonne **'Unnamed: 32'** est vide : on va la supprimer 

In [ ]:
df = df.drop(['Unnamed: 32'], axis=1)

In [ ]:
df.diagnosis.value_counts()

## Visualisations

In [ ]:
malin = df.diagnosis=='M'
benin = df.diagnosis=='B'

*jointplot* permet de visualiser dans un plan les distributions d'un couple de paramètres :

In [ ]:
sns.jointplot("perimeter_worst", "area_worst", df, kind='kde');

In [ ]:
plt.figure(figsize=(12,12))
sns.kdeplot(df.perimeter_worst, df.area_worst,  shade=True)

On peut tracer ce type de graphique avec des couleurs :

In [ ]:
plt.figure(figsize=(12,12))
sns.kdeplot(df[malin].perimeter_worst, df[malin].area_worst, cmap="Reds",  shade=True, alpha=0.3, shade_lowest=False)
sns.kdeplot(df[benin].perimeter_worst, df[benin].area_worst, cmap="Greens", shade=True, alpha=0.3, shade_lowest=False)

Les **diagrammes en boîte** (ou **boîtes à moustaches** ou **box plot**) résument quelques caractéristiques de position du caractère étudié (médiane, quartiles, minimum, maximum ou déciles). Ce diagramme est utilisé principalement pour comparer un même caractère dans deux populations de tailles différentes. Il s'agit de tracer un rectangle allant du premier quartile au troisième quartile et coupé par la médiane. On ajoute alors des segments aux extrémités menant jusqu'aux valeurs extrêmes.  
Par exemple pour la répartion des espèces selon la longueur du sépale :

In [ ]:
sns.boxplot(x="diagnosis", y="perimeter_worst", data=df)

Les **violins plots** sont similaires aux box plots, excepté qu’ils permettent de montrer la courbe de densité de probabilité des différentes valeurs. Typiquement, les violins plots présentent un marqueur pour la médiane des données et l’écart interquartile, comme dans un box plot standard.

In [ ]:
sns.violinplot(x="diagnosis", y="perimeter_worst", data=df)

*FacetGrid* permet de superposer des graphiques selon une ou plusieurs caractéristiques. On crée une structure avec *FacetGrid*, et on trace ensuite les graphiques avec *map*

In [ ]:
fig = sns.FacetGrid(df, hue="diagnosis", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "perimeter_worst", shade=True)
fig.add_legend()

On veut tracer un nuage de points selon le rayon et la texture de la tumeur, en différenciant la couleur des points selon le diagnostic :

In [ ]:
sns.lmplot(x="radius_mean", y="texture_mean", data=df, fit_reg=False, hue='diagnosis')

*pairplot* affiche les nuages de points associés à tous les couples de paramètres :

In [ ]:
#sns.pairplot(df, hue="diagnosis")

## Machine learning

On sépare le dataset en deux parties :
- un ensemble d'apprentissage (entre 70% et 90% des données), qui va permettre d'entraîner le modèle
- un ensemble de test (entre 10% et 30% des données), qui va permettre d'estimer la pertinence de la prédiction

In [ ]:
data_train = df.sample(frac=0.8, random_state=1)          # 80% des données avec frac=0.8
data_test = df.drop(data_train.index)     # le reste des données pour le test

On sépare les données d'apprentissage (*X_train*) et la cible (*y_train*, la colonnes des données *classe*)

In [ ]:
X_train = data_train.drop(['diagnosis'], axis=1)
y_train = data_train['diagnosis']
X_test = data_test.drop(['diagnosis'], axis=1)
y_test = data_test['diagnosis']

## Régression logistique

On veut prédire une variable aléatoire $Y$ à partir d'un vecteur de variables explicatives $X=(X_1,...,X_n)$
On 


La fonction logistique $\frac{e^{x}}{1+e^{x}}$ varie entre $-\infty$ et $+\infty$ pour $x$ variant entre $0$ et $1$.  
Elle est souvent utilisée pour "mapper" une probabilité et un espace réel

In [ ]:
plt.figure(figsize=(9,9))

logistique = lambda x: np.exp(x)/(1+np.exp(x))   

x_range = np.linspace(-10,10,50)       
y_values = logistique(x_range)

plt.plot(x_range, y_values, color="red")

La régression logistique consiste à trouver une fonction linéaire C(X) qui permette d'estimer la probabilité de $Y=1$ connaissant $X$ :
$$p(Y=1|X) = \frac{e^{C(X)}}{1+e^{C(X)}}$$

Autrement dit, cela revient à trouver une séparation linéaire des caractéristiques qui minimise un critère d'erreur.

Pour plus de détails, cf par exemple :  
http://eric.univ-lyon2.fr/~ricco/cours/cours/pratique_regression_logistique.pdf

On peut tracer la courbe de régression logistique pour prédire l'espèce Virginica à partir de la longueur du sépale avec la fonction *lmplot* :

On veut maintenant prédire l'espèce à partir de toutes les caractéristiques, et évaluer la qualité de cette prédiction en utilisant la régression logistique définie dans la librairie *sklearn* :

In [ ]:
from sklearn.linear_model import LogisticRegression

On entraîne le modèle de régression logistique avec *fit* :

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

On peut prédire les valeurs sur l'ensemble de test avec le modèle entraîné :

In [ ]:
y_lr = lr.predict(X_test)

## Score et matrice de confusion

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

La mesure de pertinence compte le nombre de fois où l'algorithme a fait une bonne prédiction (en pourcentage) :

In [ ]:
lr_score = accuracy_score(y_test, y_lr)
print(lr_score)

Une mesure plus fine consiste à compter le nombre de **faux positif** (valeur prédite 1 et réelle 0) et de **vrai négatif** (valeur prédite 0 et réelle 1). On utilise une **matrice de confusion** :

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, y_lr)
print(cm)

<img src="https://i.stack.imgur.com/gKyb9.png">

On peut aussi utiliser la méthode **crosstab** de **Pandas** (plutôt que la méthode confusion_matrix de sklearn) pour afficher la matrice de confusion :

In [ ]:
pd.crosstab(y_test, y_lr, rownames=['Reel'], colnames=['Prediction'], margins=True)

### Exercice  
Le résultat est-il satisfaisant ?  
Quel pourrait être le problème ?

## Arbres de décision

In [ ]:
fig = sns.FacetGrid(df, hue="diagnosis", aspect=3) # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "perimeter_worst", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(df[df.perimeter_worst>110], hue="diagnosis", aspect=3) # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "texture_mean", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(df[(df.perimeter_worst>110) & (df.texture_mean>17)], hue="diagnosis", aspect=3) # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "concave points_mean", shade=True)
fig.add_legend()

Un arbre de décision permet de faire à chaque étape un choix entre deux possibilités, pour arriver à une réponse sur les feuilles (cf. Akinator)  
<img src="https://fr.akinator.com/bundles/elokencesite/images/akitudes_670x1096/defi.png?v95">

Pour construire un arbre de décision à partir d'un ensemble d'apprentissage, on va choisir une variable qui sépare l'ensemble en deux parties les plus distinctes en fonction d'un critère. Sur les iris par exemple, on peut utiliser la largeur du pétale pour séparer l'espèce Setosa des autres.

L'indice *GINI* mesure avec quelle fréquence un élément aléatoire de l'ensemble serait mal classé si son étiquette était sélectionnée aléatoirement depuis la distribution des étiquettes dans le sous-ensemble.

In [ ]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier()
dtc.fit(X_train,y_train)
y_dtc = dtc.predict(X_test)
print(accuracy_score(y_test, y_dtc))

In [ ]:
plt.figure(figsize=(30,30))
tree.plot_tree(dtc, feature_names=X_train.columns, class_names=['benin','malin'], fontsize=14, filled=True)  

On peut modifier certains paramètres :  Le paramètre *max_depth* est un seuil sur la profondeur maximale de l’arbre. Le paramètre *min_samples_leaf* donne le nombre minimal d’échantillons dans un noeud feuille.

In [ ]:
dtc1 = tree.DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 20)
dtc1.fit(X_train,y_train)

On obtient un arbre un peu différent :

In [ ]:
plt.figure(figsize=(30,30))
tree.plot_tree(dtc1, feature_names=X_train.columns, class_names=['benin','malin'], fontsize=14, filled=True)  

In [ ]:
y_dtc1 = dtc1.predict(X_test)
print(accuracy_score(y_test, y_dtc1))

Pour plus de détails sur les arbres de décision :  
https://zestedesavoir.com/tutoriels/962/les-arbres-de-decisions/comprendre-le-concept/#1-les-origines  
http://cedric.cnam.fr/vertigo/Cours/ml2/tpArbresDecision.html  
http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/Slides/coursFM_AD-RF.pdf  

## Random forests

<img src="https://infinitescript.com/wordpress/wp-content/uploads/2016/08/Random-Forest-Example.jpg">

cf par exemple :  
https://fr.wikipedia.org/wiki/For%C3%AAt_d%27arbres_d%C3%A9cisionnels  
https://www.biostars.org/p/86981/  
https://infinitescript.com/2016/08/random-forest/

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

Et la matrice de confusion :

In [ ]:
pd.crosstab(y_test, y_rf, rownames=['Reel'], colnames=['Prediction'], margins=True)

On a un *faux positif* et deux *faux négatifs*

### Importance des caractéristiques

L'attribut *feature_importances_* renvoie un tableau du poids de chaque caractéristique dans la décision :

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)

On peut visualiser ces degrés d'importance avec un graphique à barres par exemple :

In [ ]:
plt.figure(figsize=(12,8))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), df.columns[indices])
plt.title('Importance des caracteristiques')

### Exercice : les iris

Le dataset des iris est prédéfini dans seaborn :

In [ ]:
df = sns.load_dataset("iris")

In [ ]:
df.head()

On a les informations suivantes :
- longueur du sépale (en cm)
- largeur du sépale
- longueur du pétale
- largeur du pétale
- espèce : Virginica, Setosa ou Versicolor

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQM3aH4Q3AplfE1MR3ROAp9Ok35fafmNT59ddXkdEvNdMkT8X6E">

In [ ]:
sns.pairplot(df, hue="species")

Appliquer les méthodes de visualisation et de machine learning vues précédemment sur ce dataset